<a href="https://colab.research.google.com/github/PraveenMadhava/Coursera-Client/blob/master/B2c_Extractionv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# File Upload Block

In [ ]:
# prompt: Generate a Code to upload xls file. should check file if not xls give message "Please check the file does not look like xls" And if file is xls check if the file has worksheet with name starting "scope_". show message as "Number of Worksheet found with "scope_" and "Number of Worksheet without and will not be used"

import pandas as pd
from google.colab import files

uploaded = files.upload()

scope_dataframes = {}
df_worksheetlist = [] # This will now store worksheet names

for filename in uploaded.keys():
    if not filename.endswith('.xls') and not filename.endswith('.xlsx'):
        print(f"Please check the file '{filename}' does not look like xls")
    else:
        try:
            xls = pd.ExcelFile(filename)
            scope_worksheets = [sheet_name for sheet_name in xls.sheet_names if sheet_name.lower().startswith('scope_')]
            other_worksheets = [sheet_name for sheet_name in xls.sheet_names if not sheet_name.lower().startswith('scope_')]

            print(f"Number of Worksheets found with \"scope_\": {len(scope_worksheets)}")
            print(f"Number of Worksheets without \"scope_\" and will not be used: {len(other_worksheets)}")

            for sheet_name in scope_worksheets:
                # Store only the worksheet name in the list
                df_worksheetlist.append(sheet_name)
                # You can still optionally load the dataframe into the dictionary if needed later
                # df = xls.parse(sheet_name)
                # scope_dataframes[sheet_name] = df
                print(f"Found and added worksheet name to list: {sheet_name}")


        except Exception as e:
            print(f"An error occurred while processing '{filename}': {e}")

# You can now access the list of worksheet names
# print("\nList of 'scope_' worksheet names:")
# print(df_worksheetlist)

# You can still access the dataframes from scope_dataframes dictionary if they were loaded
# print("\nDataframes loaded from 'scope_' worksheets:")
# for sheet_name, df in scope_dataframes.items():
#     print(f"\nWorksheet: {sheet_name}")
#     display(df.head())

Saving CampaignData_3.xlsx to CampaignData_3.xlsx
Number of Worksheets found with "scope_": 3
Number of Worksheets without "scope_" and will not be used: 0
Found and added worksheet name to list: scope_broadreach
Found and added worksheet name to list: scope_men
Found and added worksheet name to list: scope_AA


Generating List_Manager

In [ ]:
# prompt: Extract all Worksheet with Name starting with scope_ and add to dataframe called List_Manager with Index names as SheetName and ProcessFlag set to default 0

import pandas as pd
# Create the DataFrame List_Manager from the list of worksheet names
List_Manager = pd.DataFrame({
    'SheetName': df_worksheetlist,
    'ProcessFlag': 0  # Set default ProcessFlag to 0
})

# Set 'SheetName' as the index
List_Manager.set_index('SheetName', inplace=True)

# Display the new DataFrame
print("\nList_Manager DataFrame:")
display(List_Manager)



List_Manager DataFrame:


,ProcessFlag
SheetName,
scope_broadreach,0
scope_men,0
scope_AA,0


pick sheetname from list_manager whre processflag is 0

In [ ]:
# prompt: pick the first sheetname from list_manager where processflag is 0

# Pick the first sheetname from List_Manager where ProcessFlag is 0
first_sheet_to_process = List_Manager[List_Manager['ProcessFlag'] == 0].index.tolist()

if first_sheet_to_process:
    selected_sheet_name = first_sheet_to_process[0]
    print(f"\nSelected sheet name to process: {selected_sheet_name}")
else:
    selected_sheet_name = None
    print("\nNo sheets with ProcessFlag == 0 found.")

# You can now use the selected_sheet_name variable for further processing
# For example, if you had loaded the dataframes earlier:
# if selected_sheet_name and selected_sheet_name in scope_dataframes:
#     df_to_process = scope_dataframes[selected_sheet_name]
#     print(f"\nDataFrame for {selected_sheet_name}:")
#     display(df_to_process.head())


Selected sheet name to process: scope_broadreach


In [ ]:
# prompt: set a RunSheet=From List_Manager pick from first sheetname where ProcessFlag is 0

# Set a variable 'RunSheet' to the selected sheet name
RunSheet = selected_sheet_name

print(f"\nRunSheet variable is set to: {RunSheet}")


RunSheet variable is set to: scope_broadreach


In [ ]:
# prompt: Extract data from worksheet=RunSheet and extract data into three different dataframes  RScope_Extract , RClaims_Extract, RCombinator_Extract  and it should delete all row which is Nan an all column which is Nan

import pandas as pd
# Load the DataFrame for the selected sheet (RunSheet)
RScope_Extract = pd.DataFrame()
RClaims_Extract = pd.DataFrame()
RCombinator_Extract = pd.DataFrame()


if 'RunSheet' in locals() and RunSheet:
    # Get the filename from the uploaded dictionary (assuming only one file was uploaded)
    if uploaded:
        filename = list(uploaded.keys())[0]
        try:
            # Use the filename to load the Excel file
            xls = pd.ExcelFile(filename)
            # Parse the specific sheet into a DataFrame with no header initially to preserve all data
            df_RunSheet = xls.parse(RunSheet, header=None)
            print(f"\nFull DataFrame loaded for '{RunSheet}':")
            # display(df_RunSheet.head()) # Optional: display head of the full dataframe

            # Identify the start rows for "SCOPE", "CLAIMS & FEATURES MATRIX", and "COMBINATOR OVERVIEW"
            scope_start_row = -1
            claims_features_start = -1
            combinator_overview_start = -1
            guidelines_start = -1

            for index, row in df_RunSheet.iterrows():
                if row.astype(str).str.contains("SCOPE", case=False).any():
                    scope_start_row = index
                if row.astype(str).str.contains("CLAIMS & FEATURES MATRIX", case=False).any():
                    claims_features_start = index
                if row.astype(str).str.contains("COMBINATOR OVERVIEW", case=False).any():
                    combinator_overview_start = index
                if row.astype(str).str.contains("GUIDELINES", case=False).any():
                    guidelines_start = index


            # --- Extract and Clean Scope Data ---
            if scope_start_row != -1 and claims_features_start != -1 and claims_features_start > scope_start_row:
                temp_scope_df = df_RunSheet.iloc[scope_start_row:claims_features_start].copy()
                # Remove rows and columns with all NaN values
                temp_scope_df_cleaned = temp_scope_df.dropna(axis=0, how='all').dropna(axis=1, how='all')
                RScope_Extract = temp_scope_df_cleaned
                print("\nRScope_Extract DataFrame (cleaned):")
                display(RScope_Extract)
            else:
                print("\nCould not identify Scope section for extraction.")
                RScope_Extract = pd.DataFrame() # Ensure it's an empty DataFrame if not found

            # --- Extract and Clean Claims & Features Data ---
            if claims_features_start != -1 and combinator_overview_start != -1 and combinator_overview_start > claims_features_start:
                temp_claims_df = df_RunSheet.iloc[claims_features_start:combinator_overview_start].copy()
                 # Remove rows and columns with all NaN values
                temp_claims_df_cleaned = temp_claims_df.dropna(axis=0, how='all').dropna(axis=1, how='all')
                RClaims_Extract = temp_claims_df_cleaned
                print("\nRClaims_Extract DataFrame (cleaned):")
                display(RClaims_Extract)
            else:
                print("\nCould not identify Claims & Features section for extraction.")
                RClaims_Extract = pd.DataFrame() # Ensure it's an empty DataFrame if not found


            # --- Extract and Clean Combinator Overview Data ---
            if combinator_overview_start != -1:
                 # Assuming Combinator Overview goes until Guidelines or end of dataframe if Guidelines not found
                if guidelines_start != -1 and guidelines_start > combinator_overview_start:
                    temp_combinator_df = df_RunSheet.iloc[combinator_overview_start:guidelines_start].copy()
                else:
                     temp_combinator_df = df_RunSheet.iloc[combinator_overview_start:].copy()

                 # Remove rows and columns with all NaN values
                temp_combinator_df_cleaned = temp_combinator_df.dropna(axis=0, how='all').dropna(axis=1, how='all')
                RCombinator_Extract = temp_combinator_df_cleaned
                print("\nRCombinator_Extract DataFrame (cleaned):")
                display(RCombinator_Extract)
            else:
                print("\nCould not identify Combinator Overview section for extraction.")
                RCombinator_Extract = pd.DataFrame() # Ensure it's an empty DataFrame if not found


        except FileNotFoundError:
             print(f"\nError: File '{filename}' not found after upload.")
        except Exception as e:
            print(f"An error occurred while processing '{RunSheet}' from '{filename}': {e}")
    else:
        print("\nNo file was uploaded. Please run the file upload cell first.")
else:
    print(f"\n'RunSheet' variable is not set or is empty. Please ensure a sheet name is selected.")


Full DataFrame loaded for 'scope_AA':

RScope_Extract DataFrame (cleaned):


,0,1,2
0,SCOPE,NaN,NaN
1,Client,Native,Project for brand
2,Category,Body Wash,The category for which Drivers should be disco...
3,Geography,USA,The geographic market the campaign(s) will be ...
4,Time Period,P12M,The time period for which Drivers and associat...
5,Product Line,Regular,Product Line for campaign (Regular or Moisturi...
6,No. of Variants (client brand),6,"Total number of Variants, can be N/A if campai..."
7,Expected Posts,150 across 5 campaigns,Total number of posts across all campaigns
9,Variant List,NaN,NaN
10,Coconut & Vanilla,< Default variant,NaN



RClaims_Extract DataFrame (cleaned):


,0,1,2,3,4,5,6,7
18,CLAIMS & FEATURES MATRIX (FOR DRIVER VARIATIONS),NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Add more rows or columns as necessary,FEATURES,NaN,NaN,NaN,NaN,NaN,NaN
20,Claims,Residue-free formula,Coconut & Vanilla,Citrus & Herbal Musk,Lavender & Rose,Cucumber & Mint,Cotton & Lily,Sweet Peach & Nectar
21,Clear body wash with no dyes.,Y,Y,Y,Y,Y,Y,Y
22,Pristine transparency with no dyes.,Y,Y,Y,Y,Y,Y,Y
23,Honest ingredients with no dyes.,Y,Y,Y,Y,Y,Y,Y



RCombinator_Extract DataFrame (cleaned):


,0,1,2,3,4,5
26,COMBINATOR OVERVIEW (FOR PROMPT LOGIC),NaN,NaN,NaN,NaN,NaN
27,Add more rows or columns as necessary,COMBINATORS,NaN,NaN,NaN,NaN
28,Claims,Driver,Education,Occasions,Pay-offs,Problem Situations
29,Clear body wash with no dyes.,"Product Works Really Well, Moisturizes/Hydrate...","Benefits of using natural body wash, Harmfulne...","Before a date, For important conferences, For ...","Balanced oil production, Easier to apply make-...","After hiking or camping, After long periods of..."
30,Pristine transparency with no dyes.,"Product Works Really Well, Moisturizes/Hydrate...","Benefits of using natural body wash, Harmfulne...","Before a date, For important conferences, For ...","Balanced oil production, Easier to apply make-...","After hiking or camping, After long periods of..."
31,Honest ingredients with no dyes.,"Product Works Really Well, Moisturizes/Hydrate...","Benefits of using natural body wash, Harmfulne...","Before a date, For important conferences, For ...","Balanced oil production, Easier to apply make-...","After hiking or camping, After long periods of..."
34,GUIDELINES,NaN,NaN,NaN,NaN,NaN


In [ ]:
# prompt: Extract following  Campaign = SheetName exclude scope_ , and use RScope_Extract and In Index 0 look for Client and set Client=Index 1 value, In Index 0 Look for Category and set Category= Index 1 and In Index 0  Look Product Line and set ProductLine= Index 1 , In Index 0 looks for variant list and set for variant list = all value in index 0 below variant list till end of the row. The variant list values below index variant list should be as rows . And insert to dataframe Scope_Data

import pandas as pd
# Initialize an empty DataFrame to store the extracted scope data
Scope_Data = pd.DataFrame()

if not RScope_Extract.empty:
    # Determine the Campaign name from the RunSheet variable, excluding 'scope_'
    Campaign = RunSheet.replace('scope_', '', 1) # Replace only the first occurrence

    client = None
    category = None
    product_line = None
    variant_list_start_index = -1
    variant_list_values = []

    # Iterate through the rows of the RScope_Extract DataFrame
    for index, row in RScope_Extract.iterrows():
        # Look for Client
        if pd.notna(row.iloc[0]) and 'Client' in str(row.iloc[0]):
            if len(row) > 1 and pd.notna(row.iloc[1]):
                client = row.iloc[1]
            continue # Move to the next row

        # Look for Category
        if pd.notna(row.iloc[0]) and 'Category' in str(row.iloc[0]):
            if len(row) > 1 and pd.notna(row.iloc[1]):
                category = row.iloc[1]
            continue # Move to the next row

        # Look for Product Line
        if pd.notna(row.iloc[0]) and 'Product Line' in str(row.iloc[0]):
            if len(row) > 1 and pd.notna(row.iloc[1]):
                product_line = row.iloc[1]
            continue # Move to the next row

        # Look for Variant List and capture values below it
        if pd.notna(row.iloc[0]) and 'Variant List' in str(row.iloc[0]):
            variant_list_start_index = index
            # Start collecting values from the next row, but check if there are values in the first column
            # If there are other key-value pairs like Client, Category, etc., after Variant List, handle them
            continue # Move to the next row

        # Collect variant list values if we are after the "Variant List" row
        if variant_list_start_index != -1 and index > variant_list_start_index:
            if pd.notna(row.iloc[0]): # Only add non-NaN values from the first column
                variant_list_values.append(row.iloc[0])


    # Create the DataFrame Scope_Data
    # Check if variant_list_values is empty and handle accordingly
    if variant_list_values:
        Scope_Data = pd.DataFrame({
            'Campaign': Campaign,
            'Client': client,
            'Category': category,
            'ProductLine': product_line,
            'VariantList': variant_list_values
        })
    else:
         # If no variant list values found, create a row with other extracted info (or empty)
        Scope_Data = pd.DataFrame({
            'Campaign': [Campaign] if Campaign is not None else [None],
            'Client': [client] if client is not None else [None],
            'Category': [category] if category is not None else [None],
            'ProductLine': [product_line] if product_line is not None else [None],
            'VariantList': [None] # Or an empty string, depending on desired representation
        })


    print("\nScope_Data DataFrame:")
    display(Scope_Data)

else:
    print("\nRScope_Extract DataFrame is empty. Scope_Data will be an empty DataFrame.")
    Scope_Data = pd.DataFrame() # Ensure Scope_Data is empty if no scope data was extracted




Scope_Data DataFrame:


,Campaign,Client,Category,ProductLine,VariantList
0,AA,Native,Body Wash,Regular,Coconut & Vanilla
1,AA,Native,Body Wash,Regular,Citrus & Herbal Musk
2,AA,Native,Body Wash,Regular,Lavender & Rose
3,AA,Native,Body Wash,Regular,Cucumber & Mint
4,AA,Native,Body Wash,Regular,Cotton & Lily
5,AA,Native,Body Wash,Regular,Sweet Peach & Nectar


In [ ]:
import pandas as pd
# Initialize an empty list to store the extracted claims and features data
claims_extract_data = []

if not RClaims_Extract.empty:
    # Determine the Campaign name from the RunSheet variable, excluding 'scope_'
    # Assuming RunSheet is available from a previous step
    if 'RunSheet' in locals() and RunSheet:
        Campaign = RunSheet.replace('scope_', '', 1) # Replace only the first occurrence
    else:
        Campaign = 'Unknown Campaign' # Default if RunSheet is not available
        print("Warning: 'RunSheet' variable not found. Using 'Unknown Campaign'.")


    claims_label_row_index = -1
    claims_col_label = -1 # Use label for column access

    # Find the row and column label where "Claims" appears in RClaims_Extract
    # Search across all columns
    for index, row in RClaims_Extract.iterrows():
        for col_label, value in row.items():
            if pd.notna(value) and 'Claims' in str(value):
                claims_label_row_index = index
                claims_col_label = col_label
                break # Found the Claims label, break inner loop
        if claims_label_row_index != -1:
            break # Found the Claims label, break outer loop


    if claims_label_row_index != -1 and claims_col_label != -1:
        print(f"Found 'Claims' at row index {claims_label_row_index} and column label {claims_col_label}")

        # Extract the Claims from the column BELOW the "Claims" label
        # Use .loc with the row slice (starting from the row after the label) and the claims column label
        claims_list = RClaims_Extract.loc[claims_label_row_index + 1:, claims_col_label].dropna().tolist()

        # Extract Features from the SAME row as "Claims", starting from the column AFTER the "Claims" column
        # The features are in the columns to the right of the Claims column, in the same row as "Claims"
        # Need to find the column labels to the right of claims_col_label in the header row
        features_header_row = RClaims_Extract.loc[claims_label_row_index]
        # Select columns using their labels from the column after claims_col_label onwards
        # Need to find the position of claims_col_label in the columns to slice correctly by label
        try:
            claims_col_position = RClaims_Extract.columns.get_loc(claims_col_label)
            features_start_col_position = claims_col_position + 1
            # Get the labels of the feature columns
            feature_col_labels = RClaims_Extract.columns[features_start_col_position:].tolist()
            # Extract the feature names from the header row using the feature column labels
            feature_names_series = features_header_row.loc[feature_col_labels]
            feature_names = feature_names_series.dropna().tolist()

        except KeyError:
             print(f"Error: Could not find column label {claims_col_label} in RClaims_Extract columns.")
             feature_names = []
             feature_col_labels = []


        # Create the Claims_Extract DataFrame
        # Assuming RunSheet is available for the Campaign name
        # Campaign = RunSheet.replace('scope_', '', 1) if 'RunSheet' in locals() and RunSheet else 'Unknown Campaign' # Defined earlier


        # Iterate through the extracted claims and pair them with the features and their values
        for claim_index, claim in enumerate(claims_list):
            # Find the row index for the current claim in the claims column (claims_col_label)
            # Search in the full RClaims_Extract using .loc with the claims column label
            claim_row_match_in_full = RClaims_Extract[RClaims_Extract.loc[:, claims_col_label].astype(str).str.strip() == str(claim).strip()]

            if not claim_row_match_in_full.empty:
                claim_row_index = claim_row_match_in_full.index[0] # Get the actual row index

                # Iterate through the extracted feature names and their column labels
                for feature_index_in_list, feature_name in enumerate(feature_names):
                    # The column label for the current feature is feature_col_labels[feature_index_in_list]
                    current_feature_col_label = feature_col_labels[feature_index_in_list]


                    # Get the value from the intersection of the claim row index and feature column label
                    # Use .loc with the row index and column label
                    if current_feature_col_label in RClaims_Extract.columns: # Ensure the column label exists
                         intersection_value = RClaims_Extract.loc[claim_row_index, current_feature_col_label]

                         claims_extract_data.append({
                             'Campaign': Campaign,
                             'Claims': claim,
                             'Features': feature_name,
                             'Value': intersection_value # Include the intersection value
                         })
                    # else:
                        # Handle case where feature column label is not found (shouldn't happen if feature_col_labels are from RClaims_Extract.columns)
                        # print(f"Warning: Feature column label {current_feature_col_label} not found for feature '{feature_name}'")

            # else:
                # Handle case where claim could not be found in RClaims_Extract (shouldn't happen if claims list was built correctly)
                # print(f"Warning: Could not find row for claim '{claim}' in RClaims_Extract")


        # Create the DataFrame from the collected data
        Claims_Extract = pd.DataFrame(claims_extract_data)

        print("\nClaims_Extract DataFrame:")
        display(Claims_Extract)

    else:
        print("\nCould not find 'Claims' in RClaims_Extract.")
        Claims_Extract = pd.DataFrame(columns=['Campaign', 'Claims', 'Features', 'Value'])


else:
    print("\nRClaims_Extract DataFrame is empty.")
    Claims_Extract = pd.DataFrame(columns=['Campaign', 'Claims', 'Features', 'Value']) # Ensure an empty DataFrame is created

Found 'Claims' at row index 20 and column label 0

Claims_Extract DataFrame:


,Campaign,Claims,Features,Value
0,AA,Clear body wash with no dyes.,Residue-free formula,Y
1,AA,Clear body wash with no dyes.,Coconut & Vanilla,Y
2,AA,Clear body wash with no dyes.,Citrus & Herbal Musk,Y
3,AA,Clear body wash with no dyes.,Lavender & Rose,Y
4,AA,Clear body wash with no dyes.,Cucumber & Mint,Y
5,AA,Clear body wash with no dyes.,Cotton & Lily,Y
6,AA,Clear body wash with no dyes.,Sweet Peach & Nectar,Y
7,AA,Pristine transparency with no dyes.,Residue-free formula,Y
8,AA,Pristine transparency with no dyes.,Coconut & Vanilla,Y
9,AA,Pristine transparency with no dyes.,Citrus & Herbal Musk,Y


In [ ]:
import pandas as pd

# Initialize an empty list to store the extracted combinator data
combinators_stage_data_list = []

if 'RCombinator_Extract' in locals() and not RCombinator_Extract.empty:
    # Determine the Campaign name from the RunSheet variable, excluding 'scope_'
    # Assuming RunSheet is available from a previous step
    if 'RunSheet' in locals() and RunSheet:
        Campaign = RunSheet.replace('scope_', '', 1) # Replace only the first occurrence
    else:
        Campaign = 'Unknown Campaign' # Default if RunSheet is not available
        print("Warning: 'RunSheet' variable not found. Using 'Unknown Campaign'.")

    claims_label_row_index = -1
    claims_col_label = -1 # Use label for column access

    # Find the row and column label where "Claims" appears in RCombinator_Extract
    # Search across all columns
    for index, row in RCombinator_Extract.iterrows():
        for col_label, value in row.items():
            if pd.notna(value) and 'Claims' in str(value):
                claims_label_row_index = index
                claims_col_label = col_label
                break # Found the Claims label, break inner loop
        if claims_label_row_index != -1:
            break # Found the Claims label, break outer loop


    if claims_label_row_index != -1 and claims_col_label != -1:
        print(f"Found 'Claims' label in RCombinator_Extract at row index {claims_label_row_index} and column label {claims_col_label}")

        # Extract the Claims from the column BELOW the "Claims" label
        # Use .loc with the row slice (starting from the row after the label) and the claims column label
        claims_list = RCombinator_Extract.loc[claims_label_row_index + 1:, claims_col_label].dropna().tolist()

        # Extract Combinators (Features) from the SAME row as "Claims", starting from the column AFTER the "Claims" column
        # Need to find the column labels to the right of claims_col_label in the header row
        combinators_header_row = RCombinator_Extract.loc[claims_label_row_index]
        # Find the position of claims_col_label in the columns to slice correctly by label
        try:
            claims_col_position = RCombinator_Extract.columns.get_loc(claims_col_label)
            combinators_start_col_position = claims_col_position + 1
            # Get the labels of the combinator columns
            combinator_col_labels = RCombinator_Extract.columns[combinators_start_col_position:].tolist()
            # Extract the combinator names from the header row using the combinator column labels
            combinator_names_series = combinators_header_row.loc[combinator_col_labels]
            combinator_names = combinator_names_series.dropna().tolist()

        except KeyError:
             print(f"Error: Could not find column label {claims_col_label} in RCombinator_Extract columns.")
             combinator_names = []
             combinator_col_labels = []


        # The actual matrix data starts below the combinator header row
        matrix_data_start_row_index = claims_label_row_index + 1

        # Iterate through the extracted claims and pair them with the combinators and their intersection values (Topic)
        for claim_index, claim in enumerate(claims_list):
            # Find the row index for the current claim in the claims column (claims_col_label)
            # Search in the full RCombinator_Extract using .loc with the claims column label
            claim_row_match_in_full = RCombinator_Extract[RCombinator_Extract.loc[:, claims_col_label].astype(str).str.strip() == str(claim).strip()]

            if not claim_row_match_in_full.empty:
                claim_row_index = claim_row_match_in_full.index[0] # Get the actual row index

                # Iterate through the extracted combinator names and their column labels
                for combinator_index_in_list, combinator_name in enumerate(combinator_names):
                    # The column label for the current combinator is combinator_col_labels[combinator_index_in_list]
                    current_combinator_col_label = combinator_col_labels[combinator_index_in_list]


                    # Get the value from the intersection of the claim row index and combinator column label
                    # Use .loc with the row index and column label
                    if current_combinator_col_label in RCombinator_Extract.columns: # Ensure the column label exists
                         topic_value = RCombinator_Extract.loc[claim_row_index, current_combinator_col_label]

                         # Only include rows where the topic value is not NaN or empty
                         if pd.notna(topic_value) and str(topic_value).strip() != '':
                             combinators_stage_data_list.append({
                                 'Campaign': Campaign,
                                 'Claims': claim,
                                 'Combinators': combinator_name,
                                 'Topic': topic_value # Include the intersection value as Topic
                             })
                         # else:
                             # print(f"Skipping row for Claim '{claim}' and Combinator '{combinator_name}' due to empty Topic value.")

                    # else:
                        # Handle case where combinator column label is not found (shouldn't happen if combinator_col_labels are from RCombinator_Extract.columns)
                        # print(f"Warning: Combinator column label {current_combinator_col_label} not found for combinator '{combinator_name}'")

            # else:
                # Handle case where claim could not be found in RCombinator_Extract (shouldn't happen if claims list was built correctly)
                # print(f"Warning: Could not find row for claim '{claim}' in RCombinator_Extract")


        # Create the DataFrame from the collected data
        Combinators_stage_Dataframe = pd.DataFrame(combinators_stage_data_list)

        print("\nCombinators_stage_Dataframe:")
        display(Combinators_stage_Dataframe)

    else:
        print("\nCould not find 'Claims' label in RCombinator_Extract.")
        Combinators_stage_Dataframe = pd.DataFrame(columns=['Campaign', 'Claims', 'Combinators', 'Topic'])


else:
    print("\nRCombinator_Extract DataFrame is not available or is empty.")
    Combinators_stage_Dataframe = pd.DataFrame(columns=['Campaign', 'Claims', 'Combinators', 'Topic']) # Ensure an empty DataFrame is created

Found 'Claims' label in RCombinator_Extract at row index 28 and column label 0

Combinators_stage_Dataframe:


,Campaign,Claims,Combinators,Topic
0,AA,Clear body wash with no dyes.,Driver,"Product Works Really Well, Moisturizes/Hydrate..."
1,AA,Clear body wash with no dyes.,Education,"Benefits of using natural body wash, Harmfulne..."
2,AA,Clear body wash with no dyes.,Occasions,"Before a date, For important conferences, For ..."
3,AA,Clear body wash with no dyes.,Pay-offs,"Balanced oil production, Easier to apply make-..."
4,AA,Clear body wash with no dyes.,Problem Situations,"After hiking or camping, After long periods of..."
5,AA,Pristine transparency with no dyes.,Driver,"Product Works Really Well, Moisturizes/Hydrate..."
6,AA,Pristine transparency with no dyes.,Education,"Benefits of using natural body wash, Harmfulne..."
7,AA,Pristine transparency with no dyes.,Occasions,"Before a date, For important conferences, For ..."
8,AA,Pristine transparency with no dyes.,Pay-offs,"Balanced oil production, Easier to apply make-..."
9,AA,Pristine transparency with no dyes.,Problem Situations,"After hiking or camping, After long periods of..."


In [ ]:
# prompt: Scope_Data,Claims_Extract,Combinators_stage_Dataframe how many records each and if i join on all 3 using campaign

import pandas as pd
print(f"\nNumber of records in Scope_Data: {len(Scope_Data)}")
print(f"Number of records in Claims_Extract: {len(Claims_Extract)}")
print(f"Number of records in Combinators_stage_Dataframe: {len(Combinators_stage_Dataframe)}")

# Perform the joins based on 'Campaign'

# Join Scope_Data and Claims_Extract on 'Campaign'
# We'll use an outer join to keep all rows from both dataframes
# Note: 'Claims_Extract' also has 'Claims' and 'Features', consider which columns you want in the final merged dataframe
# For simplicity, we'll just join on 'Campaign' and keep all columns for now.
# You might need to adjust this based on your exact desired output.
merged_scope_claims = pd.merge(Scope_Data, Claims_Extract, on='Campaign', how='outer')
print("\nMerged DataFrame (Scope_Data and Claims_Extract on 'Campaign'):")
print(f"Number of records after joining Scope_Data and Claims_Extract: {len(merged_scope_claims)}")
display(merged_scope_claims.head()) # Display the head of the merged dataframe

# Join the result with Combinators_stage_Dataframe on 'Campaign'
# Combinators_stage_Dataframe has 'Claims' and 'Combinators', and 'Topic'
# Joining on 'Campaign' alone might create duplicates if Campaign appears multiple times in a dataframe.
# If you want to join based on 'Campaign' AND 'Claims', you would specify that in the 'on' parameter.
# Let's join on 'Campaign' for now as requested, but be aware of potential duplicates if Campaigns are not unique keys.
final_merged_df = pd.merge(merged_scope_claims, Combinators_stage_Dataframe, on='Campaign', how='outer', suffixes=('_claims', '_combinators'))

print("\nFinal Merged DataFrame (Scope_Data, Claims_Extract, and Combinators_stage_Dataframe on 'Campaign'):")
print(f"Number of records after joining all three dataframes: {len(final_merged_df)}")
display(final_merged_df.head()) # Display the head of the final merged dataframe


Number of records in Scope_Data: 6
Number of records in Claims_Extract: 21
Number of records in Combinators_stage_Dataframe: 15

Merged DataFrame (Scope_Data and Claims_Extract on 'Campaign'):
Number of records after joining Scope_Data and Claims_Extract: 126


,Campaign,Client,Category,ProductLine,VariantList,Claims,Features,Value
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y
1,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Coconut & Vanilla,Y
2,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Citrus & Herbal Musk,Y
3,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Lavender & Rose,Y
4,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Cucumber & Mint,Y



Final Merged DataFrame (Scope_Data, Claims_Extract, and Combinators_stage_Dataframe on 'Campaign'):
Number of records after joining all three dataframes: 1890


,Campaign,Client,Category,ProductLine,VariantList,Claims_claims,Features,Value,Claims_combinators,Combinators,Topic
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Driver,"Product Works Really Well, Moisturizes/Hydrate..."
1,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Education,"Benefits of using natural body wash, Harmfulne..."
2,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Occasions,"Before a date, For important conferences, For ..."
3,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Pay-offs,"Balanced oil production, Easier to apply make-..."
4,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Problem Situations,"After hiking or camping, After long periods of..."


In [ ]:
# prompt: Final Merged DataFrame has topic with values comma separated. I need to split comma separated and convert into rows by campaign,client,category,productline,variantlist,claims,features,value,claims_combinators,combinators and topic values split by commas into rows

# Ensure the 'Topic' column exists and is of string type before splitting
if 'Topic' in final_merged_df.columns:
    final_merged_df['Topic'] = final_merged_df['Topic'].astype(str)

    # Split the 'Topic' column by comma and expand into multiple rows
    final_merged_df_split = final_merged_df.assign(Topic=final_merged_df['Topic'].str.split(',')).explode('Topic')

    # Strip leading/trailing whitespace from the split topic values
    final_merged_df_split['Topic'] = final_merged_df_split['Topic'].str.strip()

    print("\nFinal Merged DataFrame after splitting 'Topic' into rows:")
    print(f"Number of records after splitting Topic: {len(final_merged_df_split)}")
    display(final_merged_df_split.head())
else:
    print("\n'Topic' column not found in the final merged DataFrame. Cannot split.")
    final_merged_df_split = final_merged_df.copy() # Keep the original if 'Topic' isn't there


Final Merged DataFrame after splitting 'Topic' into rows:
Number of records after splitting Topic: 21546


,Campaign,Client,Category,ProductLine,VariantList,Claims_claims,Features,Value,Claims_combinators,Combinators,Topic
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Driver,Product Works Really Well
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Driver,Moisturizes/Hydrates
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Driver,Lathers Well
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Driver,Makes My Skin Soft
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Clear body wash with no dyes.,Driver,Love The Way The Product Feels


In [ ]:
# prompt: The above code runs for one worksheet_list. After each run update worksheet_list processflag to 1 for the current runsheet. then take the next worksheet_list and repeat till all in processflag is 1 and not sheetnames are left unprocessed. By worksheet_list name as final merged dataframe is processed  add all final merged dataframe data to FinalConsolidatedData and export as csv

import pandas as pd
# Initialize an empty DataFrame to store the final consolidated data
if 'FinalConsolidatedData' not in globals():
    FinalConsolidatedData = pd.DataFrame()

# Start the loop to process worksheets based on List_Manager
while List_Manager[List_Manager['ProcessFlag'] == 0].empty is False:
    # Pick the first sheetname from List_Manager where ProcessFlag is 0
    selected_sheet_name = List_Manager[List_Manager['ProcessFlag'] == 0].index.tolist()[0]
    print(f"\n--- Processing sheet: {selected_sheet_name} ---")

    # Set the RunSheet variable for the current loop iteration
    RunSheet = selected_sheet_name

    # --- Re-run the data extraction and merging logic for the current RunSheet ---
    # Load the DataFrame for the selected sheet (RunSheet)
    RScope_Extract = pd.DataFrame()
    RClaims_Extract = pd.DataFrame()
    RCombinator_Extract = pd.DataFrame()

    if 'RunSheet' in locals() and RunSheet:
        # Get the filename from the uploaded dictionary (assuming only one file was uploaded)
        if uploaded:
            filename = list(uploaded.keys())[0]
            try:
                xls = pd.ExcelFile(filename)
                df_RunSheet = xls.parse(RunSheet, header=None)

                scope_start_row = -1
                claims_features_start = -1
                combinator_overview_start = -1
                guidelines_start = -1

                for index, row in df_RunSheet.iterrows():
                    if row.astype(str).str.contains("SCOPE", case=False).any():
                        scope_start_row = index
                    if row.astype(str).str.contains("CLAIMS & FEATURES MATRIX", case=False).any():
                        claims_features_start = index
                    if row.astype(str).str.contains("COMBINATOR OVERVIEW", case=False).any():
                        combinator_overview_start = index
                    if row.astype(str).str.contains("GUIDELINES", case=False).any():
                        guidelines_start = index


                # --- Extract and Clean Scope Data ---
                if scope_start_row != -1 and claims_features_start != -1 and claims_features_start > scope_start_row:
                    temp_scope_df = df_RunSheet.iloc[scope_start_row:claims_features_start].copy()
                    temp_scope_df_cleaned = temp_scope_df.dropna(axis=0, how='all').dropna(axis=1, how='all')
                    RScope_Extract = temp_scope_df_cleaned
                else:
                    print(f"Could not identify Scope section for '{RunSheet}'.")
                    RScope_Extract = pd.DataFrame()

                # --- Extract and Clean Claims & Features Data ---
                if claims_features_start != -1 and combinator_overview_start != -1 and combinator_overview_start > claims_features_start:
                    temp_claims_df = df_RunSheet.iloc[claims_features_start:combinator_overview_start].copy()
                    temp_claims_df_cleaned = temp_claims_df.dropna(axis=0, how='all').dropna(axis=1, how='all')
                    RClaims_Extract = temp_claims_df_cleaned
                else:
                    print(f"Could not identify Claims & Features section for '{RunSheet}'.")
                    RClaims_Extract = pd.DataFrame()

                # --- Extract and Clean Combinator Overview Data ---
                if combinator_overview_start != -1:
                    if guidelines_start != -1 and guidelines_start > combinator_overview_start:
                        temp_combinator_df = df_RunSheet.iloc[combinator_overview_start:guidelines_start].copy()
                    else:
                         temp_combinator_df = df_RunSheet.iloc[combinator_overview_start:].copy()

                    temp_combinator_df_cleaned = temp_combinator_df.dropna(axis=0, how='all').dropna(axis=1, how='all')
                    RCombinator_Extract = temp_combinator_df_cleaned
                else:
                    print(f"Could not identify Combinator Overview section for '{RunSheet}'.")
                    RCombinator_Extract = pd.DataFrame()

            except FileNotFoundError:
                 print(f"\nError: File '{filename}' not found after upload.")
                 # Skip processing this sheet if file not found
                 List_Manager.loc[RunSheet, 'ProcessFlag'] = 1 # Mark as processed (failed)
                 continue # Go to the next iteration
            except Exception as e:
                print(f"An error occurred while processing '{RunSheet}' from '{filename}': {e}")
                 # Skip processing this sheet on error
                List_Manager.loc[RunSheet, 'ProcessFlag'] = 1 # Mark as processed (failed)
                continue # Go to the next iteration
        else:
            print("\nNo file was uploaded. Please run the file upload cell first.")
             # If no file uploaded, stop processing
            break # Exit the while loop


    # --- Process extracted dataframes for the current RunSheet ---
    Scope_Data = pd.DataFrame()
    if not RScope_Extract.empty:
        Campaign = RunSheet.replace('scope_', '', 1)
        client = None
        category = None
        product_line = None
        variant_list_start_index = -1
        variant_list_values = []

        for index, row in RScope_Extract.iterrows():
            if pd.notna(row.iloc[0]) and 'Client' in str(row.iloc[0]):
                if len(row) > 1 and pd.notna(row.iloc[1]):
                    client = row.iloc[1]
                continue
            if pd.notna(row.iloc[0]) and 'Category' in str(row.iloc[0]):
                if len(row) > 1 and pd.notna(row.iloc[1]):
                    category = row.iloc[1]
                continue
            if pd.notna(row.iloc[0]) and 'Product Line' in str(row.iloc[0]):
                if len(row) > 1 and pd.notna(row.iloc[1]):
                    product_line = row.iloc[1]
                continue
            if pd.notna(row.iloc[0]) and 'Variant List' in str(row.iloc[0]):
                variant_list_start_index = index
                continue
            if variant_list_start_index != -1 and index > variant_list_start_index:
                if pd.notna(row.iloc[0]):
                    variant_list_values.append(row.iloc[0])

        if variant_list_values:
            Scope_Data = pd.DataFrame({
                'Campaign': Campaign,
                'Client': client,
                'Category': category,
                'ProductLine': product_line,
                'VariantList': variant_list_values
            })
        else:
            Scope_Data = pd.DataFrame({
                'Campaign': [Campaign] if Campaign is not None else [None],
                'Client': [client] if client is not None else [None],
                'Category': [category] if category is not None else [None],
                'ProductLine': [product_line] if product_line is not None else [None],
                'VariantList': [None]
            })
        print(f"\nExtracted Scope_Data for '{RunSheet}':")
        display(Scope_Data.head())
    else:
        print(f"\nRScope_Extract DataFrame is empty for '{RunSheet}'.")
        Scope_Data = pd.DataFrame()


    Claims_Extract = pd.DataFrame()
    claims_extract_data = []
    if not RClaims_Extract.empty:
        Campaign = RunSheet.replace('scope_', '', 1) if 'RunSheet' in locals() and RunSheet else 'Unknown Campaign'
        claims_label_row_index = -1
        claims_col_label = -1

        for index, row in RClaims_Extract.iterrows():
            for col_label, value in row.items():
                if pd.notna(value) and 'Claims' in str(value):
                    claims_label_row_index = index
                    claims_col_label = col_label
                    break
            if claims_label_row_index != -1:
                break

        if claims_label_row_index != -1 and claims_col_label != -1:
            claims_list = RClaims_Extract.loc[claims_label_row_index + 1:, claims_col_label].dropna().tolist()

            try:
                claims_col_position = RClaims_Extract.columns.get_loc(claims_col_label)
                features_start_col_position = claims_col_position + 1
                feature_col_labels = RClaims_Extract.columns[features_start_col_position:].tolist()
                features_header_row = RClaims_Extract.loc[claims_label_row_index]
                feature_names_series = features_header_row.loc[feature_col_labels]
                feature_names = feature_names_series.dropna().tolist()

            except KeyError:
                 print(f"Error: Could not find column label {claims_col_label} in RClaims_Extract columns for '{RunSheet}'.")
                 feature_names = []
                 feature_col_labels = []

            for claim_index, claim in enumerate(claims_list):
                claim_row_match_in_full = RClaims_Extract[RClaims_Extract.loc[:, claims_col_label].astype(str).str.strip() == str(claim).strip()]
                if not claim_row_match_in_full.empty:
                    claim_row_index = claim_row_match_in_full.index[0]
                    for feature_index_in_list, feature_name in enumerate(feature_names):
                        if feature_index_in_list < len(feature_col_labels): # Ensure feature_col_labels exists at this index
                            current_feature_col_label = feature_col_labels[feature_index_in_list]
                            if current_feature_col_label in RClaims_Extract.columns:
                                 intersection_value = RClaims_Extract.loc[claim_row_index, current_feature_col_label]
                                 claims_extract_data.append({
                                     'Campaign': Campaign,
                                     'Claims_Text': claim, # Rename to avoid conflict with Claims from Combinators
                                     'Features': feature_name,
                                     'Features_Value': intersection_value # Rename to avoid conflict with Value
                                 })
                        # else:
                            # print(f"Warning: feature_col_labels index mismatch for feature '{feature_name}'")

            Claims_Extract = pd.DataFrame(claims_extract_data)
            print(f"\nExtracted Claims_Extract for '{RunSheet}':")
            display(Claims_Extract.head())
        else:
            print(f"\nCould not find 'Claims' in RClaims_Extract for '{RunSheet}'.")
            Claims_Extract = pd.DataFrame(columns=['Campaign', 'Claims_Text', 'Features', 'Features_Value'])
    else:
        print(f"\nRClaims_Extract DataFrame is empty for '{RunSheet}'.")
        Claims_Extract = pd.DataFrame(columns=['Campaign', 'Claims_Text', 'Features', 'Features_Value'])


    Combinators_stage_Dataframe = pd.DataFrame()
    combinators_stage_data_list = []
    if 'RCombinator_Extract' in locals() and not RCombinator_Extract.empty:
        Campaign = RunSheet.replace('scope_', '', 1) if 'RunSheet' in locals() and RunSheet else 'Unknown Campaign'
        claims_label_row_index = -1
        claims_col_label = -1

        for index, row in RCombinator_Extract.iterrows():
            for col_label, value in row.items():
                if pd.notna(value) and 'Claims' in str(value):
                    claims_label_row_index = index
                    claims_col_label = col_label
                    break
            if claims_label_row_index != -1:
                break

        if claims_label_row_index != -1 and claims_col_label != -1:
            claims_list = RCombinator_Extract.loc[claims_label_row_index + 1:, claims_col_label].dropna().tolist()
            try:
                claims_col_position = RCombinator_Extract.columns.get_loc(claims_col_label)
                combinators_start_col_position = claims_col_position + 1
                combinator_col_labels = RCombinator_Extract.columns[combinators_start_col_position:].tolist()
                combinators_header_row = RCombinator_Extract.loc[claims_label_row_index]
                combinator_names_series = combinators_header_row.loc[combinator_col_labels]
                combinator_names = combinator_names_series.dropna().tolist()
            except KeyError:
                 print(f"Error: Could not find column label {claims_col_label} in RCombinator_Extract columns for '{RunSheet}'.")
                 combinator_names = []
                 combinator_col_labels = []

            for claim_index, claim in enumerate(claims_list):
                claim_row_match_in_full = RCombinator_Extract[RCombinator_Extract.loc[:, claims_col_label].astype(str).str.strip() == str(claim).strip()]
                if not claim_row_match_in_full.empty:
                    claim_row_index = claim_row_match_in_full.index[0]
                    for combinator_index_in_list, combinator_name in enumerate(combinator_names):
                        if combinator_index_in_list < len(combinator_col_labels): # Ensure combinator_col_labels exists at this index
                            current_combinator_col_label = combinator_col_labels[combinator_index_in_list]
                            if current_combinator_col_label in RCombinator_Extract.columns:
                                 topic_value = RCombinator_Extract.loc[claim_row_index, current_combinator_col_label]
                                 if pd.notna(topic_value) and str(topic_value).strip() != '':
                                     combinators_stage_data_list.append({
                                         'Campaign': Campaign,
                                         'Claims_Text': claim, # Use the renamed column
                                         'Combinators': combinator_name,
                                         'Topic': topic_value
                                     })
                            # else:
                                # print(f"Warning: Combinator column label {current_combinator_col_label} not found for combinator '{combinator_name}' in '{RunSheet}'")

            Combinators_stage_Dataframe = pd.DataFrame(combinators_stage_data_list)
            print(f"\nExtracted Combinators_stage_Dataframe for '{RunSheet}':")
            display(Combinators_stage_Dataframe.head())
        else:
            print(f"\nCould not find 'Claims' label in RCombinator_Extract for '{RunSheet}'.")
            Combinators_stage_Dataframe = pd.DataFrame(columns=['Campaign', 'Claims_Text', 'Combinators', 'Topic'])

    else:
        print(f"\nRCombinator_Extract DataFrame is not available or is empty for '{RunSheet}'.")
        Combinators_stage_Dataframe = pd.DataFrame(columns=['Campaign', 'Claims_Text', 'Combinators', 'Topic'])

    # --- Perform Joins for the current RunSheet's data ---
    # Join Scope_Data and Claims_Extract on 'Campaign'
    # Use an outer join and specify suffixes for overlapping columns if any
    merged_scope_claims = pd.merge(Scope_Data, Claims_Extract, on='Campaign', how='outer', suffixes=('_scope', '_claims'))
    print(f"\nMerged Scope and Claims data for '{RunSheet}':")
    display(merged_scope_claims.head())


    # Join the result with Combinators_stage_Dataframe on 'Campaign' and 'Claims_Text'
    # This assumes 'Claims_Text' should be used for joining as well
    # Use an outer join to keep all data if a claim appears in one but not the other
    # Need to handle potential column name clashes if not renamed already
    final_merged_df = pd.merge(merged_scope_claims, Combinators_stage_Dataframe, on=['Campaign', 'Claims_Text'], how='outer', suffixes=('_merged', '_combinators'))

    print(f"\nFinal Merged DataFrame for '{RunSheet}' before Topic split:")
    print(f"Number of records after joining all three dataframes for '{RunSheet}': {len(final_merged_df)}")
    display(final_merged_df.head())

    # Ensure the 'Topic' column exists and is of string type before splitting
    if 'Topic' in final_merged_df.columns:
        final_merged_df['Topic'] = final_merged_df['Topic'].astype(str)
        final_merged_df_split = final_merged_df.assign(Topic=final_merged_df['Topic'].str.split(',')).explode('Topic')
        final_merged_df_split['Topic'] = final_merged_df_split['Topic'].str.strip()
        print(f"\nFinal Merged DataFrame for '{RunSheet}' after splitting 'Topic':")
        print(f"Number of records after splitting Topic for '{RunSheet}': {len(final_merged_df_split)}")
        display(final_merged_df_split.head())
    else:
        print(f"\n'Topic' column not found in the final merged DataFrame for '{RunSheet}'. Cannot split.")
        final_merged_df_split = final_merged_df.copy() # Keep the original if 'Topic' isn't there

    # --- Append the processed data for the current RunSheet to FinalConsolidatedData ---
    FinalConsolidatedData = pd.concat([FinalConsolidatedData, final_merged_df_split], ignore_index=True)
    print(f"\nAppended data from '{RunSheet}' to FinalConsolidatedData.")
    print(f"Total records in FinalConsolidatedData: {len(FinalConsolidatedData)}")


    # --- Update ProcessFlag for the current RunSheet ---
    List_Manager.loc[RunSheet, 'ProcessFlag'] = 1
    print(f"\nUpdated ProcessFlag for '{RunSheet}' in List_Manager.")
    display(List_Manager)

# After the loop finishes (all sheets processed)
print("\n--- All sheets processed ---")

# Remove duplicate rows from the FinalConsolidatedData if any exist due to joins or errors
FinalConsolidatedData.drop_duplicates(inplace=True)
print(f"\nRemoved duplicates from FinalConsolidatedData. Final record count: {len(FinalConsolidatedData)}")


# Define the output filename
output_filename = 'FinalConsolidatedData.csv'

# Export the final consolidated data to a CSV file
try:
    FinalConsolidatedData.to_csv(output_filename, index=False)
    print(f"\nSuccessfully exported FinalConsolidatedData to '{output_filename}'")

    # Provide the link to download the file (optional)
    # from google.colab import files
    # files.download(output_filename) # Uncomment to automatically download the file

except Exception as e:
    print(f"\nAn error occurred during CSV export: {e}")




--- Processing sheet: scope_broadreach ---

Extracted Scope_Data for 'scope_broadreach':


,Campaign,Client,Category,ProductLine,VariantList
0,broadreach,Native,Body Wash,Regular,Coconut & Vanilla
1,broadreach,Native,Body Wash,Regular,Citrus & Herbal Musk
2,broadreach,Native,Body Wash,Regular,Lavender & Rose
3,broadreach,Native,Body Wash,Regular,Cucumber & Mint
4,broadreach,Native,Body Wash,Regular,Cotton & Lily



Extracted Claims_Extract for 'scope_broadreach':


,Campaign,Claims_Text,Features,Features_Value
0,broadreach,"Gentle, skin-soothing embrace.",Residue-free formula,Y
1,broadreach,"Gentle, skin-soothing embrace.",Plant-based cleansers,Y
2,broadreach,"Gentle, skin-soothing embrace.",Coconut & Vanilla,Y
3,broadreach,"Gentle, skin-soothing embrace.",Citrus & Herbal Musk,Y
4,broadreach,"Gentle, skin-soothing embrace.",Lavender & Rose,Y



Extracted Combinators_stage_Dataframe for 'scope_broadreach':


...


Final Merged DataFrame for 'scope_AA' after splitting 'Topic':
Number of records after splitting Topic for 'scope_AA': 7182


,Campaign,Client,Category,ProductLine,VariantList,Claims_Text,Features,Features_Value,Combinators,Topic
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Driver,Product Works Really Well
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Driver,Moisturizes/Hydrates
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Driver,Lathers Well
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Driver,Makes My Skin Soft
0,AA,Native,Body Wash,Regular,Coconut & Vanilla,Clear body wash with no dyes.,Residue-free formula,Y,Driver,Love The Way The Product Feels



Appended data from 'scope_AA' to FinalConsolidatedData.
Total records in FinalConsolidatedData: 59130

Updated ProcessFlag for 'scope_AA' in List_Manager.


,ProcessFlag
SheetName,
scope_broadreach,1
scope_men,1
scope_AA,1



--- All sheets processed ---

Removed duplicates from FinalConsolidatedData. Final record count: 29565

Successfully exported FinalConsolidatedData to 'FinalConsolidatedData.csv'
